In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from scipy.misc import derivative
import time
import gc
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
train = pd.read_hdf('../input/train.h5')
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,1.0,NaN,NaN,NaN,1,NaN,104,6.0,9.0,18,1438873,2019-06-07 15:32:01,8,2.135019e+09,0,2329670524,601
1,135939,893,NaN,NaN,NaN,NaN,1,NaN,19,6.0,8.0,0,1185582,2019-06-08 19:40:40,4,2.782306e+09,1,2864801071,1000
2,399254,821,NaN,760.0,NaN,360.0,1,NaN,559,NaN,8.0,0,1555716,2019-06-06 23:59:13,0,1.392806e+09,2,628911675,696
3,68983,1004,1.0,2214.0,NaN,1080.0,0,NaN,129,2.0,8.0,0,1093419,2019-06-09 09:00:12,0,3.562553e+09,3,1283809327,753
4,288999,1076,1.0,2280.0,NaN,1080.0,1,1.0,64,2.0,8.0,0,1400089,2019-06-07 08:28:13,5,2.364522e+09,4,1510695983,582


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  500000 non-null  int32  
 1   apptype     500000 non-null  int16  
 2   carrier     435093 non-null  float64
 3   dev_height  392986 non-null  float16
 4   dev_ppi     116709 non-null  float16
 5   dev_width   392989 non-null  float16
 6   label       500000 non-null  int8   
 7   lan         316720 non-null  float64
 8   media_id    500000 non-null  int16  
 9   ntt         479383 non-null  float16
 10  osv         493439 non-null  float64
 11  package     500000 non-null  int16  
 12  sid         500000 non-null  int32  
 13  timestamp   500000 non-null  object 
 14  version     500000 non-null  int64  
 15  fea_hash    499910 non-null  float64
 16  location    500000 non-null  int16  
 17  fea1_hash   500000 non-null  int64  
 18  cus_type    500000 non-null  int16  
dtypes:

In [4]:
test1 = pd.read_hdf('../input/test2.h5')
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,0,1001,1.0,640.0,NaN,360.0,1.0,29,5.0,8.0,4,1177564,2019-06-08 15:18:31,0.0,3.426912e+09,34,2694799399,411
1,0,1106,1.0,NaN,NaN,NaN,NaN,39,2.0,8.0,65,1822122,2019-06-04 21:53:18,0.0,1.331186e+09,48,4190764663,411
2,0,1001,1.0,640.0,NaN,360.0,1.0,29,2.0,6.0,4,1178302,2019-06-08 20:28:43,0.0,1.118061e+09,25,2610913319,411
3,196009,1001,1.0,2016.0,NaN,1080.0,1.0,29,6.0,7.0,4,1269787,2019-06-08 15:51:17,3.0,3.780086e+09,3,2711576615,905
4,80444,1076,1.0,2280.0,NaN,1080.0,1.0,64,2.0,9.0,0,1604520,2019-06-05 23:22:29,5.0,1.274342e+09,108,51536943,734


In [5]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  150000 non-null  int32  
 1   apptype     150000 non-null  int16  
 2   carrier     130369 non-null  float64
 3   dev_height  117568 non-null  float16
 4   dev_ppi     34910 non-null   float16
 5   dev_width   117571 non-null  float16
 6   lan         94563 non-null   float64
 7   media_id    150000 non-null  int16  
 8   ntt         143774 non-null  float16
 9   osv         148095 non-null  float64
 10  package     150000 non-null  int16  
 11  sid         150000 non-null  int32  
 12  timestamp   150000 non-null  object 
 13  version     149999 non-null  float64
 14  fea_hash    149968 non-null  float64
 15  location    150000 non-null  int16  
 16  fea1_hash   150000 non-null  int64  
 17  cus_type    150000 non-null  int16  
dtypes: float16(4), float64(5), int16(5), int32(2

In [6]:
data = pd.concat([train, test1])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 650000 entries, 0 to 149999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  650000 non-null  int32  
 1   apptype     650000 non-null  int16  
 2   carrier     565462 non-null  float64
 3   dev_height  510554 non-null  float16
 4   dev_ppi     151619 non-null  float16
 5   dev_width   510560 non-null  float16
 6   label       500000 non-null  float64
 7   lan         411283 non-null  float64
 8   media_id    650000 non-null  int16  
 9   ntt         623157 non-null  float16
 10  osv         641534 non-null  float64
 11  package     650000 non-null  int16  
 12  sid         650000 non-null  int32  
 13  timestamp   650000 non-null  object 
 14  version     649999 non-null  float64
 15  fea_hash    649878 non-null  float64
 16  location    650000 non-null  int16  
 17  fea1_hash   650000 non-null  int64  
 18  cus_type    650000 non-null  int16  
dtypes:

发现：android_id为0的有165013个，比其他的特征要高出了100多个数量级，猜测，0可能是缺失值的一种编码方式

In [8]:
data['android_id'].value_counts()[:10]

0         165013
1190         272
6964         117
16662         39
2604          38
8183          36
44716         26
43274         26
19949         24
114020        23
Name: android_id, dtype: int64

In [9]:
data['dev_height*dev_width'] = data['dev_height'] * data['dev_width']

In [10]:
data['apptype_media_id'] = data['apptype'].astype(str) + '_' + data['media_id'].astype(str)
# data['apptype_media_id_location'] = data['apptype'].astype(str) + '_' + data['media_id'].astype(str) + '_' + data['location'].astype(str)

data['apptype_osv'] = data['apptype'].astype(str) + '_' + data['osv'].astype(str)
data['apptype_osv'] = data['apptype'].astype(str) + '_' + data['package'].astype(str)

data['media_id_osv'] = data['media_id'].astype(str) + '_' + data['osv'].astype(str)
data['media_id_osv'] = data['media_id'].astype(str) + '_' + data['package'].astype(str)

data['apptype_osv'] = data['apptype'].astype(str) + '_' + data['media_id'].astype(str) + '_'+ data['osv'].astype(str)
data['apptype_osv'] = data['apptype'].astype(str) + '_' + data['media_id'].astype(str) + '_'+ data['osv'].astype(str) + '_' + data['package'].astype(str)

In [11]:
cate_cols = ['android_id', 'apptype', 'carrier', 'lan', 'media_id', 'ntt', 'osv',
             'version', 'location', 'cus_type', 'package', 'apptype_media_id']

print('There are about {} categorical features.'.format(len(cate_cols)))

There are about 12 categorical features.


In [12]:
for i in ['apptype', 'media_id', 'osv', 'package', 'apptype_media_id']:
    for j in ['dev_height', 'dev_width', 'fea1_hash']:
        df_tmp = data.groupby(i, as_index=False)[j].agg({
            '{}_{}_min'.format(i, j): 'min',
            '{}_{}_max'.format(i, j): 'max',
            '{}_{}_median'.format(i, j): 'median',
            '{}_{}_mean'.format(i, j): 'mean',
            '{}_{}_sum'.format(i, j): 'sum',
            '{}_{}_count'.format(i, j): 'count',
            '{}_{}_std'.format(i, j): 'std',
            '{}_{}_nunique'.format(i, j): 'nunique',
            '{}_{}_skew'.format(i, j): 'skew'
        })
        data = pd.merge(data, df_tmp, on=i, how='left')
        del df_tmp
        gc.collect()

In [13]:
# count编码
for col in cate_cols: 
    print(col)
    # data[col] = data[col].astype('category')
    data[col + '_cnt_code'] = data[col].map(data[col].value_counts()) 

android_id
apptype
carrier
lan
media_id
ntt
osv
version
location
cus_type
package
apptype_media_id


In [14]:
data['user_id'] = (data['package'].astype(str) + '_' + data['version'].astype(str) + '_' + data['android_id'].astype(str)
                   + '_' + data['carrier'].astype(str) + '_' + data['osv'].astype(str) + '_' + data['lan'].astype(str))

In [15]:
data['day'] = data['timestamp'].astype('datetime64').dt.day
data['hour'] = data['timestamp'].astype('datetime64').dt.hour
data['dayofweek'] = data['timestamp'].astype('datetime64').dt.dayofweek

data['time'] = (data['day'] - data['day'].min()) * 24 + data['hour']

In [16]:
train['day'] = train['timestamp'].astype('datetime64').dt.day
train['day'].unique()

array([ 7,  8,  6,  9,  3,  4,  5, 10], dtype=int64)

In [17]:
test1['day'] = test1['timestamp'].astype('datetime64').dt.day
test1['day'].unique()

array([ 8,  4,  5,  9,  6,  3,  7, 10], dtype=int64)

In [18]:
data['day'].unique()

array([ 7,  8,  6,  9,  3,  4,  5, 10], dtype=int64)

In [19]:
time_cols = ['day', 'hour', 'dayofweek', 'time']

# count编码
for col in time_cols: 
    print(col)
    data[col + '_cnt_code'] = data[col].map(data[col].value_counts()) 

day
hour
dayofweek
time


## 用户特征1
1. 用户每天在某个app上出现的次数
2. 用户每天，每小时出现的次数
3. 用户距上次出现的时间差

In [20]:
# 用户每天/每小时在某个app上出现的次数
for i in ['day', 'time']:
    print('user_id_' + i + '_media')
    data['user_id_' + i + '_media'] = data['user_id'] + '_' + data[i].astype(str) + '_' + data['media_id'].astype(str)
    dic_ = data['user_id_' + i + '_media'].value_counts().to_dict()
    data['user_id_' + i + '_media_count'] = data['user_id_' + i + '_media'].apply(lambda x: dic_[x])
    data.drop('user_id_' + i + '_media', axis=1, inplace = True)

user_id_day_media
user_id_time_media


In [21]:
# 用户每天/每小时在某个app上出现的次数
for i in ['day', 'time']:
    print('user_id_' + i + '_app')
    data['user_id_' + i + '_app'] = data['user_id'] + '_' + data[i].astype(str) + '_' + data['apptype'].astype(str)
    dic_ = data['user_id_' + i + '_app'].value_counts().to_dict()
    data['user_id_' + i + '_app_count'] = data['user_id_' + i + '_app'].apply(lambda x: dic_[x])
    data.drop('user_id_' + i + '_app', axis=1, inplace = True)

user_id_day_app
user_id_time_app


In [22]:
# 用户每天,每个小时出现的次数
for i in ['day', 'time']:
    print('user_id_' + i)
    data['user_id_' + i] = data['user_id'] + '_' + data[i].astype(str)
    dic_ = data['user_id_' + i].value_counts().to_dict()
    data['user_id_' + i + '_count'] = data['user_id_' + i].apply(lambda x: dic_[x])
    data.drop('user_id_' + i, axis=1, inplace=True)

user_id_day
user_id_time


In [23]:
# 用户本次出现距上次时间差
# data['user_to_lasttime'] = data.groupby('user_id')['time'].diff().values

In [24]:
for col in data.columns:
    if col not in ['sid', 'label', 'timestamp']: 
        if data[col].dtypes == 'O':
            print(col)
            data[col+'_labelencode'] = LabelEncoder().fit_transform(data[col].values)

apptype_media_id
apptype_osv
media_id_osv
user_id


In [25]:
# 用户每天/每小时在某个app上出现的次数
for i in ['day','time']:
    for c in ['fea_hash', 'fea1_hash']:
        print('user_id_'+ i +'_' +c)
        data['user_id_'+ i + '_' +c] = data['user_id']  + '_' + data[i].astype(str) + '_' + data[c].astype(str)
        dic_ = data['user_id_'+ i +'_' +c].value_counts().to_dict()
        data['user_id_'+ i + '_' + c + '_count'] = data['user_id_'+ i +'_' + c].apply(lambda x: dic_[x])
        data.drop('user_id_'+ i +'_' + c, axis=1,inplace = True)

user_id_day_fea_hash
user_id_day_fea1_hash
user_id_time_fea_hash
user_id_time_fea1_hash


In [26]:
from sklearn.model_selection import StratifiedKFold, KFold
from itertools import product
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=10, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode
 
        :param n_splits: the number of splits used in mean encoding
 
        :param target_type: str, 'regression' or 'classification'
 
        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """
 
        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}
 
        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None
 
        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
 
    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()
 
        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()
 
        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg(['mean', 'size'])
        col_avg_y['size'] = prior_weight_func(col_avg_y['size'])
        col_avg_y[nf_name] = col_avg_y['size'] * prior + (1 - col_avg_y['size']) * col_avg_y['mean']
        col_avg_y.drop(['size', 'mean'], axis=1, inplace=True)
 
        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values
 
        return nf_train, nf_test, prior, col_avg_y
 
    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)
 
        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new
 
    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
 
        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
 
        return X_new

In [27]:
train_data = data.iloc[:train.shape[0], :]
test1_data = data.iloc[train.shape[0]:, :]

train_data_y = train_data['label']
train_data_X = train_data.drop('label', axis=1)

test1_data_y = test1_data['label']
test1_data_X = test1_data.drop('label', axis=1)

In [28]:
ME = MeanEncoder(categorical_features=cate_cols, n_splits=5, target_type='classification', prior_weight_func=None)
train_data_X = ME.fit_transform(train_data_X, train_data_y)
test1_data_X = ME.transform(test1_data_X)

In [29]:
train_data_X_train, train_data_X_val, train_data_y_train, train_data_y_val = train_test_split(train_data_X, train_data_y,
                                                                                              random_state=2020)

In [30]:
train_cols = [i for i in train_data_X_train.columns if i not in ['sid', 'label'] and train_data_X_train[i].dtypes != 'O']

In [31]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'num_leaves': 256,
          'learning_rate': 0.05,
          'feature_fraction': 0.95,
          'bagging_fraction': 0.85,
          'bagging_freq': 5, 
          'min_data_in_leaf': 15}

In [32]:
use_cols = []
useless_cols = []

for i in train_cols:
    print(i)
    
    lgb_train = lgb.Dataset(train_data_X_train[[i]].values, train_data_y_train) 
    lgb_eval= lgb.Dataset(train_data_X_val[[i]].values, train_data_y_val, reference=lgb_train)
    lgb_test = lgb.train(params,
                         lgb_train,
                         num_boost_round=100,
                         valid_sets=[lgb_eval, lgb_train],
                         early_stopping_rounds=20,
                         verbose_eval=10)
    
    print('*' * 5)
    print(lgb_test.best_score['valid_0']['auc'])
    if lgb_test.best_score['valid_0']['auc'] > 0.52:
        use_cols.append(i)
    else:
        useless_cols.append(i)
    print('*' * 20)
    print('\n')

android_id
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.565548	valid_0's auc: 0.556563
[20]	training's auc: 0.565756	valid_0's auc: 0.556471
Early stopping, best iteration is:
[1]	training's auc: 0.565314	valid_0's auc: 0.556797
*****
0.5567967800195105
********************


apptype
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.818892	valid_0's auc: 0.81912
[20]	training's auc: 0.8189	valid_0's auc: 0.819136
[30]	training's auc: 0.8189	valid_0's auc: 0.819139
[40]	training's auc: 0.818901	valid_0's auc: 0.819132
[50]	training's auc: 0.8189	valid_0's auc: 0.819171
[60]	training's auc: 0.818901	valid_0's auc: 0.819168
[70]	training's auc: 0.818901	valid_0's auc: 0.819139
Early stopping, best iteration is:
[50]	training's auc: 0.8189	valid_0's auc: 0.819171
*****
0.8191705067485104
********************


carrier
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.543381	valid_0's 

[10]	training's auc: 0.817863	valid_0's auc: 0.818386
[20]	training's auc: 0.817869	valid_0's auc: 0.818405
[30]	training's auc: 0.817869	valid_0's auc: 0.818405
[40]	training's auc: 0.81787	valid_0's auc: 0.818392
[50]	training's auc: 0.817869	valid_0's auc: 0.818433
[60]	training's auc: 0.817869	valid_0's auc: 0.818435
[70]	training's auc: 0.81787	valid_0's auc: 0.818401
Early stopping, best iteration is:
[53]	training's auc: 0.817869	valid_0's auc: 0.818435
*****
0.81843497969882
********************


apptype_dev_height_std
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.561471	valid_0's auc: 0.56207
[20]	training's auc: 0.561471	valid_0's auc: 0.56207
Early stopping, best iteration is:
[1]	training's auc: 0.561471	valid_0's auc: 0.56207
*****
0.5620700860032154
********************


apptype_dev_height_nunique
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.81618	valid_0's auc: 0.816656
[20]	training's auc: 0.

[20]	training's auc: 0.81046	valid_0's auc: 0.810855
Early stopping, best iteration is:
[1]	training's auc: 0.81045	valid_0's auc: 0.810961
*****
0.8109613162173416
********************


apptype_fea1_hash_skew
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.816041	valid_0's auc: 0.816226
[20]	training's auc: 0.816048	valid_0's auc: 0.816243
[30]	training's auc: 0.816048	valid_0's auc: 0.816237
[40]	training's auc: 0.816048	valid_0's auc: 0.81624
Early stopping, best iteration is:
[20]	training's auc: 0.816048	valid_0's auc: 0.816243
*****
0.81624257442374
********************


media_id_dev_height_min
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.771242	valid_0's auc: 0.767542
[20]	training's auc: 0.771244	valid_0's auc: 0.767417
Early stopping, best iteration is:
[7]	training's auc: 0.771242	valid_0's auc: 0.767542
*****
0.7675421933779126
********************


media_id_dev_height_max
Training until validation

[50]	training's auc: 0.855762	valid_0's auc: 0.853732
[60]	training's auc: 0.855764	valid_0's auc: 0.853768
Early stopping, best iteration is:
[42]	training's auc: 0.85576	valid_0's auc: 0.853772
*****
0.8537716588352859
********************


media_id_fea1_hash_median
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.865101	valid_0's auc: 0.862454
[20]	training's auc: 0.865119	valid_0's auc: 0.862464
[30]	training's auc: 0.865121	valid_0's auc: 0.862471
[40]	training's auc: 0.865122	valid_0's auc: 0.862485
[50]	training's auc: 0.865122	valid_0's auc: 0.862481
[60]	training's auc: 0.865124	valid_0's auc: 0.862479
Early stopping, best iteration is:
[43]	training's auc: 0.865122	valid_0's auc: 0.862485
*****
0.8624852756572098
********************


media_id_fea1_hash_mean
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.871515	valid_0's auc: 0.868959
[20]	training's auc: 0.871527	valid_0's auc: 0.869033
[30]	training's

Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.514904	valid_0's auc: 0.514986
[20]	training's auc: 0.514904	valid_0's auc: 0.514986
Early stopping, best iteration is:
[1]	training's auc: 0.514904	valid_0's auc: 0.514986
*****
0.5149856327609975
********************


osv_dev_width_nunique
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.766947	valid_0's auc: 0.765808
[20]	training's auc: 0.766947	valid_0's auc: 0.765808
Early stopping, best iteration is:
[1]	training's auc: 0.766947	valid_0's auc: 0.765808
*****
0.7658075270804243
********************


osv_dev_width_skew
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.5	valid_0's auc: 0.5
[20]	training's auc: 0.5	valid_0's auc: 0.5
Early stopping, best iteration is:
[1]	training's auc: 0.5	valid_0's auc: 0.5
*****
0.5
********************


osv_fea1_hash_min
Training until validation scores don't improve for 20 rounds
[10]	train

[10]	training's auc: 0.65848	valid_0's auc: 0.654823
[20]	training's auc: 0.65848	valid_0's auc: 0.654823
[30]	training's auc: 0.65848	valid_0's auc: 0.654824
[40]	training's auc: 0.65848	valid_0's auc: 0.654823
[50]	training's auc: 0.658481	valid_0's auc: 0.654837
[60]	training's auc: 0.658481	valid_0's auc: 0.654838
Early stopping, best iteration is:
[46]	training's auc: 0.658481	valid_0's auc: 0.654838
*****
0.6548379676285473
********************


package_dev_width_mean
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.799868	valid_0's auc: 0.796933
[20]	training's auc: 0.799872	valid_0's auc: 0.796938
[30]	training's auc: 0.799878	valid_0's auc: 0.796953
[40]	training's auc: 0.799881	valid_0's auc: 0.796965
[50]	training's auc: 0.799882	valid_0's auc: 0.79695
Early stopping, best iteration is:
[39]	training's auc: 0.799881	valid_0's auc: 0.796966
*****
0.7969656178340679
********************


package_dev_width_sum
Training until validation score

[10]	training's auc: 0.588878	valid_0's auc: 0.586141
[20]	training's auc: 0.588898	valid_0's auc: 0.586165
Early stopping, best iteration is:
[1]	training's auc: 0.588811	valid_0's auc: 0.586227
*****
0.5862269203207904
********************


apptype_media_id_dev_height_count
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.836228	valid_0's auc: 0.833708
[20]	training's auc: 0.836244	valid_0's auc: 0.83369
[30]	training's auc: 0.836244	valid_0's auc: 0.833704
Early stopping, best iteration is:
[13]	training's auc: 0.836234	valid_0's auc: 0.833718
*****
0.8337184124001212
********************


apptype_media_id_dev_height_std
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.592149	valid_0's auc: 0.590267
[20]	training's auc: 0.592167	valid_0's auc: 0.590282
Early stopping, best iteration is:
[1]	training's auc: 0.59216	valid_0's auc: 0.590291
*****
0.5902913199920737
********************


apptype_media_id_dev_height

Early stopping, best iteration is:
[29]	training's auc: 0.871846	valid_0's auc: 0.869854
*****
0.8698536831942331
********************


apptype_media_id_fea1_hash_std
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.858435	valid_0's auc: 0.855928
[20]	training's auc: 0.858463	valid_0's auc: 0.855891
[30]	training's auc: 0.858465	valid_0's auc: 0.855897
[40]	training's auc: 0.858469	valid_0's auc: 0.855883
Early stopping, best iteration is:
[23]	training's auc: 0.85845	valid_0's auc: 0.855978
*****
0.8559784917180557
********************


apptype_media_id_fea1_hash_nunique
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.737482	valid_0's auc: 0.734947
[20]	training's auc: 0.737482	valid_0's auc: 0.734942
Early stopping, best iteration is:
[4]	training's auc: 0.737456	valid_0's auc: 0.735032
*****
0.7350321532816897
********************


apptype_media_id_fea1_hash_skew
Training until validation scores don't improve 

Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.519684	valid_0's auc: 0.518698
[20]	training's auc: 0.519691	valid_0's auc: 0.518787
[30]	training's auc: 0.519675	valid_0's auc: 0.518698
Early stopping, best iteration is:
[19]	training's auc: 0.519691	valid_0's auc: 0.518787
*****
0.5187870304161608
********************


dayofweek_cnt_code
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.511259	valid_0's auc: 0.509348
[20]	training's auc: 0.511259	valid_0's auc: 0.509348
Early stopping, best iteration is:
[1]	training's auc: 0.511172	valid_0's auc: 0.509589
*****
0.5095888394200728
********************


time_cnt_code
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.526735	valid_0's auc: 0.519472
[20]	training's auc: 0.526892	valid_0's auc: 0.520078
[30]	training's auc: 0.526922	valid_0's auc: 0.520239
[40]	training's auc: 0.526931	valid_0's auc: 0.520211
Early stopping, best iter

[10]	training's auc: 0.54374	valid_0's auc: 0.540146
[20]	training's auc: 0.543779	valid_0's auc: 0.540379
[30]	training's auc: 0.543783	valid_0's auc: 0.540165
Early stopping, best iteration is:
[17]	training's auc: 0.54374	valid_0's auc: 0.540394
*****
0.540394079852787
********************


carrier_pred_1.0
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.54374	valid_0's auc: 0.540146
[20]	training's auc: 0.543779	valid_0's auc: 0.540379
[30]	training's auc: 0.543783	valid_0's auc: 0.540165
Early stopping, best iteration is:
[17]	training's auc: 0.54374	valid_0's auc: 0.540394
*****
0.540394079852787
********************


lan_pred_0.0
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.678026	valid_0's auc: 0.67474
[20]	training's auc: 0.678026	valid_0's auc: 0.67474
[30]	training's auc: 0.677971	valid_0's auc: 0.674593
Early stopping, best iteration is:
[14]	training's auc: 0.678026	valid_0's auc: 0.67474
*****
0.

In [33]:
print(use_cols)

['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'osv', 'package', 'version', 'location', 'fea1_hash', 'cus_type', 'dev_height*dev_width', 'apptype_dev_height_min', 'apptype_dev_height_max', 'apptype_dev_height_median', 'apptype_dev_height_mean', 'apptype_dev_height_sum', 'apptype_dev_height_count', 'apptype_dev_height_std', 'apptype_dev_height_nunique', 'apptype_dev_width_min', 'apptype_dev_width_max', 'apptype_dev_width_median', 'apptype_dev_width_mean', 'apptype_dev_width_sum', 'apptype_dev_width_count', 'apptype_dev_width_std', 'apptype_dev_width_nunique', 'apptype_fea1_hash_min', 'apptype_fea1_hash_max', 'apptype_fea1_hash_median', 'apptype_fea1_hash_mean', 'apptype_fea1_hash_sum', 'apptype_fea1_hash_count', 'apptype_fea1_hash_std', 'apptype_fea1_hash_nunique', 'apptype_fea1_hash_skew', 'media_id_dev_height_min', 'media_id_dev_height_max', 'media_id_dev_height_median', 'media_id_dev_height_mean', 'media_id_dev_height_sum', 'media

In [34]:
print(useless_cols)

['fea_hash', 'apptype_dev_height_skew', 'apptype_dev_width_skew', 'media_id_dev_height_skew', 'media_id_dev_width_skew', 'osv_dev_height_sum', 'osv_dev_height_std', 'osv_dev_height_skew', 'osv_dev_width_sum', 'osv_dev_width_std', 'osv_dev_width_skew', 'package_dev_height_skew', 'package_dev_width_skew', 'apptype_media_id_dev_height_skew', 'apptype_media_id_dev_width_skew', 'day', 'hour', 'dayofweek', 'day_cnt_code', 'hour_cnt_code', 'dayofweek_cnt_code', 'user_id_day_fea_hash_count', 'user_id_time_fea_hash_count']


In [35]:
lgb_train = lgb.Dataset(train_data_X_train[use_cols], train_data_y_train)
lgb_eval = lgb.Dataset(train_data_X_val[use_cols], train_data_y_val, reference=lgb_train)

In [36]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'binary_logloss',
          'num_leaves': 256,
          'learning_rate': 0.01,
          'feature_fraction': 0.95,
          'bagging_fraction': 0.85,
          'bagging_freq': 5, 
          'min_data_in_leaf':15}

In [37]:
print('Start training...')

gbm_val_1 = lgb.train(params,
                      lgb_train,
                      num_boost_round=2000,
                      valid_sets=[lgb_train, lgb_eval],
                      early_stopping_rounds=50,
                      verbose_eval=200)

Start training...
Training until validation scores don't improve for 50 rounds
[200]	training's binary_logloss: 0.312528	valid_1's binary_logloss: 0.318215
[400]	training's binary_logloss: 0.28245	valid_1's binary_logloss: 0.296268
[600]	training's binary_logloss: 0.270031	valid_1's binary_logloss: 0.29396
[800]	training's binary_logloss: 0.260155	valid_1's binary_logloss: 0.293347
[1000]	training's binary_logloss: 0.251663	valid_1's binary_logloss: 0.293063
Early stopping, best iteration is:
[1099]	training's binary_logloss: 0.247829	valid_1's binary_logloss: 0.293012


In [38]:
train_data_X_val['prob'] = gbm_val_1.predict(train_data_X_val[use_cols])
train_data_X_val['pred'] = np.where(train_data_X_val['prob'] > 0.5, 1, 0)

acc = np.round(accuracy_score(train_data_y_val, train_data_X_val['pred']), 4)
auc = roc_auc_score(train_data_y_val, train_data_X_val['prob'])

print('acc: ', acc)
print('auc: ', auc)

acc:  0.8886
auc:  0.9435833759133305


In [39]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'num_leaves': 256,
          'learning_rate': 0.01,
          'feature_fraction': 0.95,
          'bagging_fraction': 0.85,
          'bagging_freq': 5, 
          'min_data_in_leaf': 15}

In [40]:
lgb_train = lgb.Dataset(train_data_X[use_cols], train_data_y)   

print('Start training...')

gbm_1  = lgb.train(params,
                   lgb_train,
                   num_boost_round=gbm_val_1.best_iteration + 20)
print('Done')

Start training...
Done


In [41]:
test1['label'] = np.where(gbm_1.predict(test1_data_X[use_cols]) > 0.5, 1, 0)

test1[['sid', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), acc), index=False)